In [1]:
using Plots
using DiffEqFlux, OrdinaryDiffEq, Flux, Optim
include("/Users/piotrsokol/Documents/RNNODE.jl/src/rnn_ode.jl")
import DiffEqSensitivity: ischeckpointing, isconcretetype
using Zygote
using LaTeXStrings
using MLDataUtils
#ENV["PYTHON"] = "/Users/piotrsokol/anaconda3/envs/bortho/bin/python"
#using Pkg; Pkg.build("PyCall")
using PyCall

# Testing (random) copy memory implementations

### The first step to ensure consistency between the discrete time and continuous time implementation is to use the python code to generate a dataset

In [2]:
py"""
import sys
sys.path.insert(0, "/Users/piotrsokol/Documents/block-orthogonal/src/")
"""
generate_rcm = pyimport("data_utils")["_generate_random_copy_memory"]
# _generate_random_copy_memory(min_lag, max_lag, num:int, set:str, slen:int=10)

generate_cm() = pyimport("data_utils")["_generate_copy_memory"](100,Int(1e4),:train)
# _generate_copy_memory(T:int, num: int, set:str)


generate_cm (generic function with 1 method)

In [4]:
pyimport("data_utils")["_generate_copy_memory"](100,Int(1e4),:train)

([4 7 … 8 8; 5 2 … 8 8; … ; 1 0 … 8 8; 4 2 … 8 8], [8 8 … 1 3; 8 8 … 2 1; … ; 8 8 … 6 1; 8 8 … 2 5])

In [3]:
a,b = generate_cm()

([4 7 … 8 8; 5 2 … 8 8; … ; 1 0 … 8 8; 4 2 … 8 8], [8 8 … 1 3; 8 8 … 2 1; … ; 8 8 … 6 1; 8 8 … 2 5])

In [4]:
min_lag = 100
max_lag = 120
n_train = Int(1e4)
n_test = Int(1e3)
n_valid = Int(1e3)
slen = 10

input_size=9

9

In [5]:
a,b =generate_cm(100,n_train,"train")
cpt = ConstantInterpolationFixedGrid(a)

1200120-element ConstantInterpolation{Array{Int64,2},Array{Int64,1},Symbol,true,Int64}:
   4
   5
   6
   1
   2
   0
   3
   5
   2
   7
   1
   3
   7
   ⋮
 108
 109
 110
 111
 112
 113
 114
 115
 116
 117
 118
 119

In [6]:
cpt = ConstantInterpolationFixedGrid(a)

1200120-element ConstantInterpolation{Array{Int64,2},Array{Int64,1},Symbol,true,Int64}:
   4
   5
   6
   1
   2
   0
   3
   5
   2
   7
   1
   3
   7
   ⋮
 108
 109
 110
 111
 112
 113
 114
 115
 116
 117
 118
 119

In [7]:
onehot(labels_raw) = convertlabel(LabelEnc.OneOfK, labels_raw,
                                    LabelEnc.NativeLabels(collect(0:input_size-1)))

onehot (generic function with 1 method)

In [10]:
cpt

100-element Array{Int64,1}:
 4
 5
 6
 1
 2
 0
 3
 5
 2
 7
 1
 3
 7
 ⋮
 2
 4
 7
 4
 6
 2
 5
 2
 2
 1
 0
 2